# Libraries Intallation

In [ ]:
# !pip install datasets
# !pip install sentencepiece
# !pip install transformers datasets
# !pip install transformers[torch]
# !pip install accelerate
# !pip install accelerate>=0.20.1
# !pip install huggingface_hub
# !pip install kaggle
# !pip install tensorflow
# !pip install scikit-learn
# !pip install nvidia-pyindex
# !pip install nvidia-tensorflow
# !pip install tweet-preprocessor
# !python3 -m pip install tensorflow[and-cuda]
# !pip3 install pandas
!{sys.executable} -m pip install

# Import Statements

In [1]:
import os
import pandas as pd
import math
import tensorflow as tf
from sklearn.metrics import f1_score, accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import preprocessor as p
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import string
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from datasets import DatasetDict

2023-11-17 12:03:29.787422: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 12:03:29.787523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 12:03:29.787546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 12:03:29.794056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jamie/miniconda3/envs/tfgpu/lib/python3.10/s

# Set ENV Variables

In [ ]:
os.environ['KAGGLE_USERNAME'] = "evangeloskatsadouros"
os.environ['KAGGLE_KEY'] = "cd793dfe3db43b66e0c59264b92465af"

# Functions

In [2]:
def preprocess_tweet(batch):
    # tweet = batch["text"]
    # p.set_options(p.OPT.URL, p.OPT.EMOJI)
    # tweet = p.clean(tweet)
    # tweet = tweet.lower()

    # tokenizer = TweetTokenizer()
    # tokens = tokenizer.tokenize(tweet)
    # # Remove special characters and punctuation
    # tokens = [token for token in tokens if token not in string.punctuation]
    # # Remove stop words
    # stop_words = set(stopwords.words('english'))
    # tokens = [token for token in tokens if token not in stop_words]
    # # Stemming using Porter Stemmer
    # stemmer = PorterStemmer()
    # tokens = [stemmer.stem(token) for token in tokens]
    # # Rejoin the tokens to form the preprocessed text
    # preprocessed_text = ' '.join(tokens)
    
    return {'text':  batch["clean_text"], 'label': batch["category"]}


In [ ]:
# Change label values -1 to 0 (for negative tweets), 0 to 1 (for neutral tweets) and 1 to 2 (for positive tweets)
def transform_labels(labels):
    transformed_labels = []
    for label in labels:
        if label==-1:
            transformed_labels.append(0)
        elif label==0:
            transformed_labels.append(1)
        else:
            transformed_labels.append(2)

    return transformed_labels
    

In [3]:
def tokenize(batch):
    return roberta_tokenizer(batch["text"], padding="max_length", truncation=True, max_length=140)

In [4]:
# Define your custom metrics function
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids

    # Calculate F1 score
    f1 = f1_score(labels, predictions.argmax(axis=1), average='weighted')

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions.argmax(axis=1))

    return {"f1": f1, "accuracy": accuracy}

# Deprecated

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=100)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# Download Kaggle's Sentiment140 dataset (creaadits to https://www.kaggle.com/kazanova)
!kaggle datasets download -d kazanova/sentiment140
!unzip sentiment140.zip

column_names = ["target", "id", "date", "flag", "user", "text"]
dataset_df = pd.read_csv("training.1600000.processed.noemoticon.csv", names=column_names, encoding='latin-1', index_col=False)
dataset_df = dataset_df.sample(frac = 1)

# Split dataset into train and test lists
total_items = len(dataset_df)
train_point = math.floor(total_items*0.8)
eval_point = math.floor((total_items-train_point)/2)+train_point

train_texts = [preprocess_tweet(item) for item in list(dataset_df["text"])[0:train_point]]
train_targets = transform_labels(list(dataset_df["target"])[0:train_point])
eval_texts = [preprocess_tweet(item) for item in list(dataset_df["text"])[train_point:eval_point]]
eval_targets = transform_labels(list(dataset_df["target"])[train_point:eval_point])
test_texts = [preprocess_tweet(item) for item in list(dataset_df["text"])[eval_point:]]
test_targets = transform_labels(list(dataset_df["target"])[eval_point:])

# Load tokeanizer and train properties
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
batch_size = 64

train_dataset = SentimentDataset(train_texts, train_targets, roberta_tokenizer, 100)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataset = SentimentDataset(eval_texts, eval_targets, roberta_tokenizer, 100)

# Implementation

In [5]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
dataset = load_dataset("MaNaN-3/twitter_sentiment_analysis").shuffle(seed=42)
condition_to_remove = lambda example: (example['clean_text'] != None and example['category'] != None)
dataset = dataset.filter(condition_to_remove)
dataset = dataset.map(preprocess_tweet, remove_columns=['clean_text', 'category'], batched=True)
dataset = dataset.map(tokenize, batched=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 162969/162969 [00:30<00:00, 5261.97 examples/s]


In [6]:
dataset_train_eval = dataset['train'].train_test_split(test_size=0.1, seed=42)
dataset_train_test = dataset_train_eval['test'].train_test_split(test_size=0.5, seed=42)


final_dataset = DatasetDict({
    'train': dataset_train_eval['train'],
    'valid': dataset_train_test['train'],
    'test': dataset_train_test['test']
})

In [15]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=64,
    num_train_epochs=2,  # Adjust the number of training epochs
    evaluation_strategy="steps",
    save_total_limit=3,
    save_steps=1000,  # Adjust the number of steps before saving the model
    eval_steps=500,
    load_best_model_at_end=True,
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,
    train_dataset=final_dataset['train'],
    eval_dataset=final_dataset['valid'],
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,F1,Accuracy
500,0.374500,0.228861,0.926039,0.925994
1000,0.207000,0.159692,0.953200,0.953240
1500,0.164100,0.145140,0.957162,0.957045
2000,0.142500,0.134880,0.962662,0.962445
2500,0.119800,0.127636,0.965740,0.965758
3000,0.099900,0.104151,0.970547,0.970545
3500,0.086700,0.090298,0.975186,0.975209
4000,0.080700,0.084782,0.975725,0.975700
4500,0.071700,0.086448,0.977001,0.977050


TrainOutput(global_step=4584, training_loss=0.14829644129122324, metrics={'train_runtime': 1890.3747, 'train_samples_per_second': 155.178, 'train_steps_per_second': 2.425, 'total_flos': 2.110465612773504e+16, 'train_loss': 0.14829644129122324, 'epoch': 2.0})

In [20]:
trainer.evaluate(final_dataset['test'])

{'eval_loss': 0.07872962951660156,
 'eval_f1': 0.9766015398456451,
 'eval_accuracy': 0.9765615412934102,
 'eval_runtime': 20.6568,
 'eval_samples_per_second': 394.495,
 'eval_steps_per_second': 49.33,
 'epoch': 2.0}

In [21]:
set(final_dataset['train']['label'])

{0, 1, 2}